## Weight and Biases 
- DL, ML 실험을 원활히 지원하기 위한 상용도구(돈내야함 기본적인거는 그냥 사용 가능)
- 협업, code versioning, 실험 결과 기록 등 제공
- MLOps의 대표적 도구

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BoostcampAITech/lecture-note-python-basics-for-ai/blob/main/codes/pytorch/07_torch-study/visualization/wandb.ipynb)

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.8 MB 36.0 MB/s 
     |████████████████████████████████| 162 kB 57.3 MB/s 
     |████████████████████████████████| 181 kB 63.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 158 kB 71.4 MB/s 
     |████████████████████████████████| 157 kB 65.0 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |████████████████████████████████| 157 kB 65.4 MB/s 
     |████████████████████████████████| 157 kB 68.2 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |████████████████████████████████| 157 kB 73.2 MB/s 
     |████████████████████████████████| 157 kB 71.5 MB/s 
     |████████████████████████████████| 156 kB 77.3 MB/s 


In [3]:
import wandb
wandb.init(project="test-project", entity='shinunjune')

In [4]:
import torch
import torchvision
from torchvision import datasets, models, transforms

import torch.nn as nn
import torch.optim as optim
import os


import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## print out the pytorch version used (1.31 at the time of this tutorial)
print(torch.__version__)

1.12.1+cu113


In [5]:
DATA_PATH = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [6]:
import urllib
import os
import shutil
from zipfile import ZipFile

urllib.request .urlretrieve(DATA_PATH, "hymenoptera_data.zip")

with ZipFile("hymenoptera_data.zip", 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()


In [7]:
## configure root folder on your gdrive
data_dir = "./hymenoptera_data"

## custom transformer to flatten the image tensors
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        result = torch.reshape(img, self.new_size)
        return result

## transformations used to standardize and normalize the datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
}

## load the correspoding folders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

## load the entire dataset; we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],
                                            batch_size=len(image_datasets['train']),
                                            shuffle=True)

In [8]:
class MyCNNModel(nn.Module):
    def __init__(self):
        super(MyCNNModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [9]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.01

In [10]:

model = MyCNNModel()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [11]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

config : Hyperparameter 결정해줌

In [15]:
EPOCHS = 50
BATCH_SIZE = 128
LEARNING_RATE = 0.001

config={"epochs": EPOCHS, "batch_size": BATCH_SIZE, "learning_rate" : LEARNING_RATE}

In [16]:
# config : dict 타입으로 넣어줌
wandb.init(project="test-project", config=config)
# dict type 아니면 아래처럼 따로 넣어줄 수 있음
# wandb.config.batch_size = BATCH_SIZE
# wandb.config.learning_rate = LEARNING_RATE
# config={"epochs": EPOCHS, "batch_size": BATCH_SIZE, "learning_rate" : LEARNING_RATE}

for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_dataset:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        optimizer.zero_grad()        
        y_pred = model(X_batch)
               
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        
    train_loss = epoch_loss/len(train_dataset)
    train_acc = epoch_acc/len(train_dataset)
    print(f'Epoch {e+0:03}: | Loss: {train_loss:.5f} | Acc: {train_acc:.3f}')
    # 기록하고 싶은 값.
    # tensorboard의 add~ 랑 동일
    wandb.log({'accuracy': train_acc, 'loss': train_loss})


accuracy,▁▂▂▂▂▂▂▅▂▁▃▅▅▆▆▃▄▃▆▄▅▇▇▇▇▆▇▇▇▇▇▇▇▇█▇█▇█▇
loss,▁▇▂█▃▄▃▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,73.0
loss,0.52162


Epoch 001: | Loss: 0.48319 | Acc: 78.000
Epoch 002: | Loss: 0.54395 | Acc: 74.000
Epoch 003: | Loss: 0.48777 | Acc: 75.000
Epoch 004: | Loss: 0.50091 | Acc: 73.000
Epoch 005: | Loss: 0.50209 | Acc: 74.000
Epoch 006: | Loss: 0.49963 | Acc: 73.000
Epoch 007: | Loss: 0.49432 | Acc: 73.000
Epoch 008: | Loss: 0.48729 | Acc: 74.000
Epoch 009: | Loss: 0.49780 | Acc: 76.000
Epoch 010: | Loss: 0.47802 | Acc: 76.000
Epoch 011: | Loss: 0.48076 | Acc: 76.000
Epoch 012: | Loss: 0.46055 | Acc: 75.000
Epoch 013: | Loss: 0.46718 | Acc: 76.000
Epoch 014: | Loss: 0.47364 | Acc: 76.000
Epoch 015: | Loss: 0.45067 | Acc: 77.000
Epoch 016: | Loss: 0.48985 | Acc: 77.000
Epoch 017: | Loss: 0.46614 | Acc: 74.000
Epoch 018: | Loss: 0.45952 | Acc: 76.000
Epoch 019: | Loss: 0.46316 | Acc: 77.000
Epoch 020: | Loss: 0.47801 | Acc: 77.000
Epoch 021: | Loss: 0.46800 | Acc: 78.000
Epoch 022: | Loss: 0.46741 | Acc: 78.000
Epoch 023: | Loss: 0.43832 | Acc: 77.000
Epoch 024: | Loss: 0.43690 | Acc: 81.000
Epoch 025: | Los

KeyboardInterrupt: ignored